In [ ]:
import streamlit as st

In [3]:
# from streamlit_cropper import st_cropper

2022-06-09 22:24:45.294 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [4]:
%%writefile app.py
import streamlit as st
@st.experimental_singleton
def initial():
    import cv2
    import dlib
    import numpy as np
    return cv2, dlib, np
cv2, dlib, np = initial()
img_file_buffer = st.camera_input("Take a picture")
labels = ['normal', 'mild', 'severe']
if img_file_buffer:
    bytes_data = img_file_buffer.getvalue()
    img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)
    'Photo taken'
    @st.experimental_singleton
    def dlib_objs():
        fd = dlib.get_frontal_face_detector()
        sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
        return fd, sp
    fd, sp = dlib_objs()
    gray, img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = fd(img)
    if faces:
        st.balloons()
        st.success('Its Working')
        points = sp(gray, faces[0])
        cx, cy = [i.x for i in points.parts()[36:42]], [i.y for i in points.parts()[36:42]]
        x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
        mask = np.zeros(img.shape, dtype="uint8")
        mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
        right_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
        cx, cy = [i.x for i in points.parts()[42:48]], [i.y for i in points.parts()[42:48]]
        x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
        mask = np.zeros(img.shape, dtype="uint8")
        mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
        left_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
        col1, col2 = st.columns(2)
        col1.header('Left Eye')
        col1.image(left_eye, width = 300)
        col2.header('Right Eye')
        col2.image(right_eye, width = 300)
        from tensorflow.keras.models import model_from_json
        with open('model.json', 'r') as f:
            js = f.read()
        model = model_from_json(js)
        model.load_weights('weights.h5')
        preds = model.predict(np.array([cv2.resize(left_eye, (224, 224)), cv2.resize(right_eye, (224, 224))]))
        col1.write('Result - ' + labels[np.argmax(preds[0])])
        col2.write('Result - ' + labels[np.argmax(preds[1])])
    else:
        st.error('Where\'s your face?')

Overwriting app.py


In [19]:
%%writefile app.py
import streamlit as st
def check_password():
    """Returns `True` if the user had a correct password."""

    def password_entered():
        """Checks whether a password entered by the user is correct."""
        if (
            st.session_state["username"] in st.secrets["passwords"]
            and st.session_state["password"]
            == st.secrets["passwords"][st.session_state["username"]]
        ):
            st.session_state["password_correct"] = True
            del st.session_state["password"]  # don't store username + password
            del st.session_state["username"]
        else:
            st.session_state["password_correct"] = False

    if "password_correct" not in st.session_state:
        # First run, show inputs for username + password.
        st.text_input("Username", on_change=password_entered, key="username")
        st.text_input(
            "Password", type="password", on_change=password_entered, key="password"
        )
        return False
    elif not st.session_state["password_correct"]:
        # Password not correct, show input + error.
        st.text_input("Username", on_change=password_entered, key="username")
        st.text_input(
            "Password", type="password", on_change=password_entered, key="password"
        )
        st.error("😕 User not known or password incorrect")
        return False
    else:
        # Password correct.
        return True
if not st.session_state.get('guest', 0):
    chk = check_password()
    st.checkbox(label = 'Guest Login', key = 'guest', value = st.session_state.get('guest', 0))
if st.session_state['guest'] or (chk and st.session_state.get('username', 0) == 'user'):
    @st.experimental_singleton
    def initial():
        import cv2
        import dlib
        import numpy as np
        return cv2, dlib, np
    cv2, dlib, np = initial()
    st.sidebar.image('I5.png')
   
    st.sidebar.radio(label = 'Photo Options', options = ['Camera', 'Upload'], index = 0, key = 'upload')
    if st.session_state['upload'] == 'Upload':
        img_file_buffer = st.file_uploader('Upload a picture')
    else:
        img_file_buffer = st.camera_input('Take a picture')
    labels = ['normal', 'mild', 'severe']
    if img_file_buffer:
        bytes_data = img_file_buffer.getvalue()
        img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)
        'Photo taken'
        @st.experimental_singleton
        def dlib_objs():
            fd = dlib.get_frontal_face_detector()
            sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
            return fd, sp
        fd, sp = dlib_objs()
        gray, img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = fd(img)
        if faces:
            st.success('Face Detected.')
            points = sp(gray, faces[0])
            cx, cy = [i.x for i in points.parts()[36:42]], [i.y for i in points.parts()[36:42]]
            x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
            mask = np.zeros(img.shape, dtype="uint8")
            mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
            right_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
            cx, cy = [i.x for i in points.parts()[42:48]], [i.y for i in points.parts()[42:48]]
            x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
            mask = np.zeros(img.shape, dtype="uint8")
            mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
            left_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
            col1, col2 = st.columns(2)
            col1.header('Left Eye')
            col1.image(left_eye, width = 300)
            col2.header('Right Eye')
            col2.image(right_eye, width = 300)
            from tensorflow.keras.models import model_from_json
            with open('model.json', 'r') as f:
                js = f.read()
            model = model_from_json(js)
            model.load_weights('weights.h5')
            preds = model.predict(np.array([cv2.resize(left_eye, (224, 224)), cv2.resize(right_eye, (224, 224))]))
            col1.write('Result - ' + labels[np.argmax(preds[0])])
            col2.write('Result - ' + labels[np.argmax(preds[1])])
        else:
            st.error('Face Detection Failed')
    elif chk and st.session_state.get('username', 0) == 'doctor':
        st.image('I5.png')
        

Overwriting app.py


In [1]:
%%writefile app.py
import streamlit as st

@st.experimental_singleton
def initial():
    import cv2
    import dlib
    import numpy as np
    import joblib
    import xgboost
    return cv2, dlib, np, joblib, xgboost
cv2, dlib, np, joblib, xgboost = initial()
st.subheader('Welcome to Ikshana')
st.image('I5.png')
###
with st.expander('Additional Information'):
#     st.text('Instructions for usage')
    st.text('1.Please take an image that covers your entire face')
    st.text('2.Take the picture in good lighting setup')
    st.text('3.Avoid using flash while taking the picture')
###
st.selectbox(label = 'Photo Options', options = ['Camera', 'Upload'], key = 'upload')
if st.session_state['upload'] == 'Upload':
    img_file_buffer = st.file_uploader('Upload a picture')
else:
    img_file_buffer = st.camera_input('Take a picture')
labels = ['normal', 'mild', 'severe']
if img_file_buffer:
    bytes_data = img_file_buffer.getvalue()
    img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)
#     'Photo taken'
    @st.experimental_singleton
    def dlib_objs():
        fd = dlib.get_frontal_face_detector()
        sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
        return fd, sp
    fd, sp = dlib_objs()
    gray, img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = fd(img)
    if faces:
        st.success('Face Detected.')
        points = sp(gray, faces[0])
        cx, cy = [i.x for i in points.parts()[36:42]], [i.y for i in points.parts()[36:42]]
        x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
        mask = np.zeros(img.shape, dtype="uint8")
        mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
        right_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
        cx, cy = [i.x for i in points.parts()[42:48]], [i.y for i in points.parts()[42:48]]
        x, y, radius = np.mean(cx, dtype = int), np.mean(cy, dtype = int), (max(cx) - min(cx)) // 2
        mask = np.zeros(img.shape, dtype="uint8")
        mask = cv2.circle(mask, center = (x, y), radius = radius, thickness = -1, color = (1, 1, 1))
        left_eye = (img * mask)[y - radius:y + radius, x - radius:x + radius]
        col1, col2 = st.columns(2)
        col1.header('Left Eye')
        col1.image(left_eye, width = 200)
        col2.header('Right Eye')
        col2.image(right_eye, width = 200)
        from tensorflow.keras.models import model_from_json
        with open('vgg.json', 'r') as f:
            js = f.read()
        model = model_from_json(js)
        model.load_weights('vgg_weights.h5')
        xg = joblib.load('xgb.joblib')
#         with open('xgb.pkl', 'rb') as f:
#             xg = pickle.load(f)
#         preds = model.predict(np.array([cv2.resize(left_eye, (224, 224)), cv2.resize(right_eye, (224, 224))]))
        pr = model.predict(np.array([cv2.resize(left_eye, (224, 224)), cv2.resize(right_eye, (224, 224))]))
        preds = xg.predict(pr)
        label = ['No Cataract Detected.', 'Mild Cataract Detected.', 'Severe Cataract Detected.\nReach out to an eye doctor soon.']
        col1.info(label[np.argmax(preds[0])])
        col2.info(label[np.argmax(preds[1])])
        st.warning('Ensure Quality for Accurate Results.')
    else:
        st.error('Face Detection Failed. Please Try Again!!!')
    

Overwriting app.py


In [6]:
%%writefile requirements.txt
dlib-bin
opencv-python-headless
tensorflow

Overwriting requirements.txt


In [ ]:
!streamlit run app.py

In [49]:
%%writefile main.py
import streamlit as st
@st.experimental_singleton
def initial():
    import cv2
    import dlib
    import numpy as np
    return cv2, dlib, np
cv2, dlib, np = initial()
# st.sidebar.image('I0.png')

Writing main.py


In [1]:
import pickle

In [4]:
!pip pickle version

ERROR: unknown command "pickle"

